## About the CIFAR-10 dataset

[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) (classification of small images across 10 distinct classes---airplane, automobile, bird, cat, deer, dog, frog, horse, ship & truck).


![](cifar_10.png)

The CIFAR-10 dataset contains 50000 training examples and 10000 test examples. Each example is a $32 \times 32 \times 32$ pixels coloured image.

## Loading and pre-processing CIFAR-10 dataset



In [2]:
#%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import itertools
import os
os.environ['KERAS_BACKEND']="theano"
import theano

from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from sklearn.utils import shuffle
from skimage import feature

(X_train_img, y_train), (X_test_img, y_test) = cifar10.load_data() # fetch CIFAR-10 data # dirname="."??
print(X_train_img.shape)
print(X_test_img.shape)
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Shuffle examples
X_train_img, y_train = shuffle(X_train_img, y_train)
X_test_img, y_test = shuffle(X_test_img, y_test)

plt.figure()
plt.imshow(X_train_img[0])
plt.show()
print(class_names[int(y_train[0])])
plt.figure()
plt.imshow(X_train_img[1])
plt.show()
print(class_names[int(y_train[1])])
plt.figure()
plt.imshow(X_train_img[2])
plt.show()
print(class_names[int(y_train[2])])

num_train, depth, height, width = X_train_img.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test_img.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

# Produce a [0;1] normalized "image" for potential input into a convNet
X_train_img_norm = X_train_img.astype('float32') 
X_test_img_norm = X_test_img.astype('float32')
X_train_img_norm /= np.max(X_train_img_norm) # Normalise data to [0, 1] range
X_test_img_norm /= np.max(X_train_img_norm) # Normalise data to [0, 1] range

# Rasterize normalized images into 1D vectors for potential input of raw image into standard ML algo such as kNN, MLP, SVM, etc
X_train_vectNorm = X_train_img_norm.reshape(num_train,height*width*depth)
X_test_vectNorm = X_test_img_norm.reshape(num_test,height*width*depth)

print(X_train_vectNorm.shape)
print(X_test_vectNorm.shape)
print(X_train_vectNorm[1:5])


# Compute HOG feature as potential BETTER input into standard ML algo such as kNN, MLP, SVM, etc

Hog_train = []
for img in X_train_img :
    hog, hog_image = skimage.feature.hog(img, orientations=9, pixels_per_cell=(8, 8), 
                                    cells_per_block=(3, 3), block_norm=None, 
                                    visualise=True, transform_sqrt=False, feature_vector=True)
    Hog_train.append(hog)

# TO BE CORRECTED/FINALIZED!...


(50000, 32, 32, 3)
(10000, 32, 32, 3)
truck
ship
frog
(50000, 3072)
(10000, 3072)
[[ 0.25490198  0.29803923  0.27843139 ...,  0.41568628  0.41960785
   0.40000001]
 [ 0.25098041  0.1882353   0.16078432 ...,  0.17647059  0.09019608
   0.09019608]
 [ 0.14509805  0.21176471  0.11372549 ...,  0.31764707  0.37254903
   0.27450982]
 [ 0.6156863   0.68235296  0.80000001 ...,  0.70980394  0.73333335
   0.78823531]]


# Une fonction pour afficher graphiquement les matrices de confusion

In [5]:
#%matplotlib inline
import matplotlib.pyplot as plt
import itertools

# Definition for the function that plot the confusion matrix
# See : http://www.sci-kit.org/
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Baseline method: kNN on raw normalized pixels

In [4]:
from sklearn import neighbors
from sklearn.metrics import confusion_matrix

clf = neighbors.KNeighborsClassifier(n_neighbors=5)
print(y_train.shape)
clf.fit(X_train_vectNorm[:30000], y_train.ravel()[:30000]) # utiliser toutes les données conduit à une Memory Error

y_test_pred_knn = clf.predict(X_test_vectNorm)


cnf_matrix_knn = confusion_matrix(y_test, y_test_pred_knn)
print(cnf_matrix_knn)

np.set_printoptions(precision=2)
plt.figure()
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plot_confusion_matrix(cnf_matrix_knn, classes=class_names, normalize=True, title='Normalized confusion matrix')
plt.show()

(50000, 1)
[[996   0   4   0   0   0   0   0   0   0]
 [992   0   8   0   0   0   0   0   0   0]
 [991   0   9   0   0   0   0   0   0   0]
 [989   0  11   0   0   0   0   0   0   0]
 [990   0  10   0   0   0   0   0   0   0]
 [992   0   8   0   0   0   0   0   0   0]
 [993   0   6   0   0   0   0   0   1   0]
 [982   0  18   0   0   0   0   0   0   0]
 [990   0  10   0   0   0   0   0   0   0]
 [995   0   5   0   0   0   0   0   0   0]]


NameError: name 'plot_confusion_matrix' is not defined

### Remarques sur les résultats :

La matrice de confusion est très mauvaise : toutes les images sont vues comme appartenant à la première classe. Comparer les pixels des images n'est pas une méthode très pertinente pour classifier les images. Par ailleurs, l'algorithme des plus proches voisins est très lent à s'exécuter, et utilise beaucoup de mémoire : nous avons dû limiter la taille du jeu d'entraînement pour ne pas avoir d'erreur de mémoire.

## Random Forests on raw normalized pixels

In [ ]:
#Random Forests

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Create and train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None,
                             min_samples_split=2, min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0, max_features='auto', 
                             max_leaf_nodes=None, min_impurity_decrease=1e-07, bootstrap=True, 
                             oob_score=False, n_jobs=1, random_state=None, 
                             verbose=0, warm_start=False, class_weight=None)
clf = clf.fit(X_train_vectNorm, y_train.ravel())

print("n_estimators=", clf.n_estimators, " max_depth=",clf.max_depth,
      "max_features=", clf.max_features)
print(clf)

# accuracy :
score = clf.score(X_test_vectNorm, y_test)
print("Accuracy (on test set) = ", score)

# resultats et matrice de confusion
y_true, y_pred_rf = y_test, clf.predict(X_test_vectNorm)
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
print( classification_report(y_true, y_pred_rf, target_names=class_names) )

print("\n CONFUSION MATRIX")
cnf_matrix_rf=confusion_matrix(y_true, y_pred_rf)
print( cnf_matrix_rf )
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix_rf, classes=class_names, normalize=True, title='Normalized confusion matrix')
plt.show()